## Create tables for:
- [COUNTY_NAME] 4 Counties - 4 tables (can use loop) - GvR, CvN
- [MPO] By MPO - WFRC only & WFRC+MAG  2 tables - GvR, CvN
- [CITY_AREA] For each City Area - Many tables (loop required) - GvR, CvN
- [CENTER_TYPE] By Center Type - city, urban, neighborhood, town, employment  5 tables - RVG
- [INFILL1990] in/out of Infill 1990 boundary - Filter to WFRC only -  1 table - RVG, CvN

In [23]:
import os
import pandas as pd
import glob

# show all columns
pd.options.display.max_columns = None

In [24]:
outputs = os.path.join(r'.\Outputs', 'Growth_Summaries')
if not os.path.exists(outputs):
    os.makedirs(outputs)

In [25]:
# Parcel Equivalency Table
eq = pd.read_csv(r".\Inputs\parcel_eq_v3.csv")
centers_eq_ids = eq[eq['CENTER_NAME'].isna() == False]['parcel_id'].to_list()
infill_eq_ids = eq[eq['INFILL1990']==1]['parcel_id'].to_list()

# REMM project older
remm_folder = r"E:\Projects\REMM2_For_Python3_Internal_Use"
name = 'joshpc'

In [26]:
pm_folder = os.path.join(remm_folder, 'REMMRun\\Progression_Metrics')
csvs = glob.glob(os.path.join(pm_folder, 'run_*_year_*_parcel_progression_metrics.csv'))
csvs = [csv for csv in csvs if 'base' not in csv]

# By County

In [27]:
county_outputs = os.path.join(outputs, 'County')
if not os.path.exists(county_outputs):
    os.makedirs(county_outputs)

counties = list(set(eq[eq['COUNTY_NAME'].isna()==False]['COUNTY_NAME'].to_list()))
counties

['UTAH', 'WEBER', 'SALT LAKE', 'DAVIS']

In [28]:
for county in counties:
     
     summary = pd.DataFrame()
     for csv in csvs:
          df = pd.read_csv(csv)
          df = df.merge(eq, on='parcel_id', how='left')
          df = df[df['COUNTY_NAME']== county].copy()

          year = os.path.basename(csv).split('_')[3]

          #--------------------------------------
          # Centered(C) vs Noncentered (NC)
          #--------------------------------------

          # separate into centered and noncentered
          C = df[df['parcel_id'].isin(centers_eq_ids)==True].copy()
          NC = df[df['parcel_id'].isin(centers_eq_ids)==False].copy()

          # new jobs (centered)
          C_gov_edu_added = C['jobs_gov_edu_added'].sum()
          C_jobs_health_added = C['jobs_health_added'].sum()
          C_jobs_accom_food_added = C['jobs_accom_food_added'].sum()
          C_jobs_manuf_added = C['jobs_manuf_added'].sum()
          C_jobs_office_added = C['jobs_office_added'].sum()
          C_jobs_other_added = C['jobs_other_added'].sum()
          C_jobs_retail_added = C['jobs_retail_added'].sum()
          C_jobs_wholesale_added = C['jobs_wholesale_added'].sum()

          C_new_retail_jobs = C_jobs_accom_food_added + C_jobs_retail_added
          C_new_industrial_jobs = C_jobs_manuf_added + C_jobs_wholesale_added
          C_new_office_jobs = C_jobs_other_added + C_jobs_office_added + C_jobs_health_added + C_gov_edu_added
          C_new_jobs = C_new_retail_jobs + C_new_industrial_jobs + C_new_office_jobs

          # new jobs (non-centered)
          NC_gov_edu_added = NC['jobs_gov_edu_added'].sum()
          NC_jobs_health_added = NC['jobs_health_added'].sum()
          NC_jobs_accom_food_added = NC['jobs_accom_food_added'].sum()
          NC_jobs_manuf_added = NC['jobs_manuf_added'].sum()
          NC_jobs_office_added = NC['jobs_office_added'].sum()
          NC_jobs_other_added = NC['jobs_other_added'].sum()
          NC_jobs_retail_added = NC['jobs_retail_added'].sum()
          NC_jobs_wholesale_added = NC['jobs_wholesale_added'].sum()

          NC_new_retail_jobs = NC_jobs_accom_food_added + NC_jobs_retail_added
          NC_new_industrial_jobs = NC_jobs_manuf_added + NC_jobs_wholesale_added
          NC_new_office_jobs = NC_jobs_other_added + NC_jobs_office_added + NC_jobs_health_added + NC_gov_edu_added
          NC_new_jobs = NC_new_retail_jobs + NC_new_industrial_jobs + NC_new_office_jobs

          # filter to new residential
          C_new_sf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_sf']==1))]
          C_new_mf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_mf']==1))]
          NC_new_sf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_sf']==1))]
          NC_new_mf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_mf']==1))]

          # get the totals & counts
          C_new_sf_units = C_new_sf['residential_units'].sum()
          C_new_mf_units = C_new_mf['residential_units'].sum()
          NC_new_sf_units = NC_new_sf['residential_units'].sum()
          NC_new_mf_units = NC_new_mf['residential_units'].sum()
          C_new_units = C_new_sf_units + C_new_mf_units
          NC_new_units = NC_new_sf_units + NC_new_mf_units

          #---------------------------
          # Greenfield(G) vs Redev(R)
          #---------------------------

          # separate into Greenfield and Redev
          G = df[df['was_developed']==1].copy()
          R = df[df['was_redeveloped']==1].copy()

          # new jobs (Greenfield)
          G_gov_edu_added = G['jobs_gov_edu_added'].sum()
          G_jobs_health_added = G['jobs_health_added'].sum()
          G_jobs_accom_food_added = G['jobs_accom_food_added'].sum()
          G_jobs_manuf_added = G['jobs_manuf_added'].sum()
          G_jobs_office_added = G['jobs_office_added'].sum()
          G_jobs_other_added = G['jobs_other_added'].sum()
          G_jobs_retail_added = G['jobs_retail_added'].sum()
          G_jobs_wholesale_added = G['jobs_wholesale_added'].sum()

          G_new_retail_jobs = G_jobs_accom_food_added + G_jobs_retail_added
          G_new_industrial_jobs = G_jobs_manuf_added + G_jobs_wholesale_added
          G_new_office_jobs = G_jobs_other_added + G_jobs_office_added + G_jobs_health_added + G_gov_edu_added
          G_new_jobs = G_new_retail_jobs + G_new_industrial_jobs + G_new_office_jobs

          # new jobs (non-Redev)
          R_gov_edu_added = R['jobs_gov_edu_added'].sum()
          R_jobs_health_added = R['jobs_health_added'].sum()
          R_jobs_accom_food_added = R['jobs_accom_food_added'].sum()
          R_jobs_manuf_added = R['jobs_manuf_added'].sum()
          R_jobs_office_added = R['jobs_office_added'].sum()
          R_jobs_other_added = R['jobs_other_added'].sum()
          R_jobs_retail_added = R['jobs_retail_added'].sum()
          R_jobs_wholesale_added = R['jobs_wholesale_added'].sum()

          R_new_retail_jobs = R_jobs_accom_food_added + R_jobs_retail_added
          R_new_industrial_jobs = R_jobs_manuf_added + R_jobs_wholesale_added
          R_new_office_jobs = R_jobs_other_added + R_jobs_office_added + R_jobs_health_added + R_gov_edu_added
          R_new_jobs = R_new_retail_jobs + R_new_industrial_jobs + R_new_office_jobs

          # filter to new residential
          G_new_sf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_sf']==1))]
          G_new_mf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_mf']==1))]
          R_new_sf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_sf']==1))]
          R_new_mf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_mf']==1))]

          # get the totals & counts
          G_new_sf_units = G_new_sf['residential_units'].sum()
          G_new_mf_units = G_new_mf['residential_units'].sum()
          R_new_sf_units = R_new_sf['residential_units'].sum()
          R_new_mf_units = R_new_mf['residential_units'].sum()
          G_new_units = G_new_sf_units + G_new_mf_units
          R_new_units = R_new_sf_units + R_new_mf_units


          # append current data to final dataframe 
          d = {
               'year': [year],

               'C_new_res_units': [C_new_units], 'NC_new_res_units': [NC_new_units], 
               'C_new_sf_units': [C_new_sf_units], 'NC_new_sf_units': [NC_new_sf_units],
               'C_new_mf_units': [C_new_mf_units], 'NC_new_mf_units': [NC_new_mf_units],
               'C_new_jobs':[C_new_jobs], 'NC_new_jobs':[NC_new_jobs],
               'C_new_retail_jobs':[C_new_retail_jobs], 'NC_new_retail_jobs':[NC_new_retail_jobs], 
               'C_new_industrial_jobs':[C_new_industrial_jobs],'NC_new_industrial_jobs':[NC_new_industrial_jobs], 
               'C_new_office_jobs':[C_new_office_jobs], 'NC_new_office_jobs':[NC_new_office_jobs],

               'G_new_res_units': [G_new_units], 'R_new_res_units': [R_new_units], 
               'G_new_sf_units': [G_new_sf_units], 'R_new_sf_units': [R_new_sf_units],
               'G_new_mf_units': [G_new_mf_units], 'R_new_mf_units': [R_new_mf_units],
               'G_new_jobs':[G_new_jobs], 'R_new_jobs':[R_new_jobs],
               'G_new_retail_jobs':[G_new_retail_jobs], 'R_new_retail_jobs':[R_new_retail_jobs], 
               'G_new_industrial_jobs':[G_new_industrial_jobs],'R_new_industrial_jobs':[R_new_industrial_jobs], 
               'G_new_office_jobs':[G_new_office_jobs], 'R_new_office_jobs':[R_new_office_jobs]
               }

          row = pd.DataFrame(data=d)
          summary = pd.concat([summary, row]).astype(int)

          # export
          summary = summary[summary['year'] != 2019]
          summary.to_csv(os.path.join(county_outputs, f'{county}_County_Growth_{name}.csv'), index=False)

# By MPO

In [29]:
mpo_outputs = os.path.join(outputs, 'MPO')
if not os.path.exists(mpo_outputs):
    os.makedirs(mpo_outputs)

wfrc_eq = eq[eq['MPO'] == 'WFRC']['parcel_id'].to_list()
wfrc_mag_eq = eq['parcel_id'].to_list()

mpos = [['WFRC', wfrc_eq],['WFRC_MAG', wfrc_mag_eq]]

In [30]:
for mpo in mpos:

     summary = pd.DataFrame()
     for csv in csvs:
          df = pd.read_csv(csv)
          df = df.merge(eq, on='parcel_id', how='left')
          df = df[df['parcel_id'].isin(mpo[1]) == True].copy()

          year = os.path.basename(csv).split('_')[3]

          #--------------------------------------
          # Centered(C) vs Noncentered (NC)
          #--------------------------------------

          # separate into centered and noncentered
          C = df[df['parcel_id'].isin(centers_eq_ids)==True].copy()
          NC = df[df['parcel_id'].isin(centers_eq_ids)==False].copy()

          # new jobs (centered)
          C_gov_edu_added = C['jobs_gov_edu_added'].sum()
          C_jobs_health_added = C['jobs_health_added'].sum()
          C_jobs_accom_food_added = C['jobs_accom_food_added'].sum()
          C_jobs_manuf_added = C['jobs_manuf_added'].sum()
          C_jobs_office_added = C['jobs_office_added'].sum()
          C_jobs_other_added = C['jobs_other_added'].sum()
          C_jobs_retail_added = C['jobs_retail_added'].sum()
          C_jobs_wholesale_added = C['jobs_wholesale_added'].sum()

          C_new_retail_jobs = C_jobs_accom_food_added + C_jobs_retail_added
          C_new_industrial_jobs = C_jobs_manuf_added + C_jobs_wholesale_added
          C_new_office_jobs = C_jobs_other_added + C_jobs_office_added + C_jobs_health_added + C_gov_edu_added
          C_new_jobs = C_new_retail_jobs + C_new_industrial_jobs + C_new_office_jobs

          # new jobs (non-centered)
          NC_gov_edu_added = NC['jobs_gov_edu_added'].sum()
          NC_jobs_health_added = NC['jobs_health_added'].sum()
          NC_jobs_accom_food_added = NC['jobs_accom_food_added'].sum()
          NC_jobs_manuf_added = NC['jobs_manuf_added'].sum()
          NC_jobs_office_added = NC['jobs_office_added'].sum()
          NC_jobs_other_added = NC['jobs_other_added'].sum()
          NC_jobs_retail_added = NC['jobs_retail_added'].sum()
          NC_jobs_wholesale_added = NC['jobs_wholesale_added'].sum()

          NC_new_retail_jobs = NC_jobs_accom_food_added + NC_jobs_retail_added
          NC_new_industrial_jobs = NC_jobs_manuf_added + NC_jobs_wholesale_added
          NC_new_office_jobs = NC_jobs_other_added + NC_jobs_office_added + NC_jobs_health_added + NC_gov_edu_added
          NC_new_jobs = NC_new_retail_jobs + NC_new_industrial_jobs + NC_new_office_jobs

          # filter to new residential
          C_new_sf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_sf']==1))]
          C_new_mf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_mf']==1))]
          NC_new_sf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_sf']==1))]
          NC_new_mf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_mf']==1))]

          # get the totals & counts
          C_new_sf_units = C_new_sf['residential_units'].sum()
          C_new_mf_units = C_new_mf['residential_units'].sum()
          NC_new_sf_units = NC_new_sf['residential_units'].sum()
          NC_new_mf_units = NC_new_mf['residential_units'].sum()
          C_new_units = C_new_sf_units + C_new_mf_units
          NC_new_units = NC_new_sf_units + NC_new_mf_units

          #---------------------------
          # Greenfield(G) vs Redev(R)
          #---------------------------

          # separate into Greenfield and Redev
          G = df[df['was_developed']==1].copy()
          R = df[df['was_redeveloped']==1].copy()

          # new jobs (Greenfield)
          G_gov_edu_added = G['jobs_gov_edu_added'].sum()
          G_jobs_health_added = G['jobs_health_added'].sum()
          G_jobs_accom_food_added = G['jobs_accom_food_added'].sum()
          G_jobs_manuf_added = G['jobs_manuf_added'].sum()
          G_jobs_office_added = G['jobs_office_added'].sum()
          G_jobs_other_added = G['jobs_other_added'].sum()
          G_jobs_retail_added = G['jobs_retail_added'].sum()
          G_jobs_wholesale_added = G['jobs_wholesale_added'].sum()

          G_new_retail_jobs = G_jobs_accom_food_added + G_jobs_retail_added
          G_new_industrial_jobs = G_jobs_manuf_added + G_jobs_wholesale_added
          G_new_office_jobs = G_jobs_other_added + G_jobs_office_added + G_jobs_health_added + G_gov_edu_added
          G_new_jobs = G_new_retail_jobs + G_new_industrial_jobs + G_new_office_jobs

          # new jobs (non-Redev)
          R_gov_edu_added = R['jobs_gov_edu_added'].sum()
          R_jobs_health_added = R['jobs_health_added'].sum()
          R_jobs_accom_food_added = R['jobs_accom_food_added'].sum()
          R_jobs_manuf_added = R['jobs_manuf_added'].sum()
          R_jobs_office_added = R['jobs_office_added'].sum()
          R_jobs_other_added = R['jobs_other_added'].sum()
          R_jobs_retail_added = R['jobs_retail_added'].sum()
          R_jobs_wholesale_added = R['jobs_wholesale_added'].sum()

          R_new_retail_jobs = R_jobs_accom_food_added + R_jobs_retail_added
          R_new_industrial_jobs = R_jobs_manuf_added + R_jobs_wholesale_added
          R_new_office_jobs = R_jobs_other_added + R_jobs_office_added + R_jobs_health_added + R_gov_edu_added
          R_new_jobs = R_new_retail_jobs + R_new_industrial_jobs + R_new_office_jobs

          # filter to new residential
          G_new_sf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_sf']==1))]
          G_new_mf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_mf']==1))]
          R_new_sf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_sf']==1))]
          R_new_mf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_mf']==1))]

          # get the totals & counts
          G_new_sf_units = G_new_sf['residential_units'].sum()
          G_new_mf_units = G_new_mf['residential_units'].sum()
          R_new_sf_units = R_new_sf['residential_units'].sum()
          R_new_mf_units = R_new_mf['residential_units'].sum()
          G_new_units = G_new_sf_units + G_new_mf_units
          R_new_units = R_new_sf_units + R_new_mf_units


          # append current data to final dataframe 
          d = {
               'year': [year],

               'C_new_res_units': [C_new_units], 'NC_new_res_units': [NC_new_units], 
               'C_new_sf_units': [C_new_sf_units], 'NC_new_sf_units': [NC_new_sf_units],
               'C_new_mf_units': [C_new_mf_units], 'NC_new_mf_units': [NC_new_mf_units],
               'C_new_jobs':[C_new_jobs], 'NC_new_jobs':[NC_new_jobs],
               'C_new_retail_jobs':[C_new_retail_jobs], 'NC_new_retail_jobs':[NC_new_retail_jobs], 
               'C_new_industrial_jobs':[C_new_industrial_jobs],'NC_new_industrial_jobs':[NC_new_industrial_jobs], 
               'C_new_office_jobs':[C_new_office_jobs], 'NC_new_office_jobs':[NC_new_office_jobs],

               'G_new_res_units': [G_new_units], 'R_new_res_units': [R_new_units], 
               'G_new_sf_units': [G_new_sf_units], 'R_new_sf_units': [R_new_sf_units],
               'G_new_mf_units': [G_new_mf_units], 'R_new_mf_units': [R_new_mf_units],
               'G_new_jobs':[G_new_jobs], 'R_new_jobs':[R_new_jobs],
               'G_new_retail_jobs':[G_new_retail_jobs], 'R_new_retail_jobs':[R_new_retail_jobs], 
               'G_new_industrial_jobs':[G_new_industrial_jobs],'R_new_industrial_jobs':[R_new_industrial_jobs], 
               'G_new_office_jobs':[G_new_office_jobs], 'R_new_office_jobs':[R_new_office_jobs]
               }

          row = pd.DataFrame(data=d)
          summary = pd.concat([summary, row]).astype(int)

          # export
          summary = summary[summary['year'] != 2019]
          summary.to_csv(os.path.join(mpo_outputs, f'{mpo[0]}_Growth_{name}.csv'), index=False)

# By City Area

In [31]:
city_area_outputs = os.path.join(outputs, 'City_Area')
if not os.path.exists(city_area_outputs):
    os.makedirs(city_area_outputs)

cities = list(set(eq[eq['CITY_AREA'].isna()==False]['CITY_AREA'].to_list()))

In [33]:
len(cities)

95

In [32]:
for city in cities:
     
     summary = pd.DataFrame()
     for csv in csvs:
          df = pd.read_csv(csv)
          df = df.merge(eq, on='parcel_id', how='left')
          df = df[df['CITY_AREA']== city].copy()

          year = os.path.basename(csv).split('_')[3]

          #--------------------------------------
          # Centered(C) vs Noncentered (NC)
          #--------------------------------------

          # separate into centered and noncentered
          C = df[df['parcel_id'].isin(centers_eq_ids)==True].copy()
          NC = df[df['parcel_id'].isin(centers_eq_ids)==False].copy()

          # new jobs (centered)
          C_gov_edu_added = C['jobs_gov_edu_added'].sum()
          C_jobs_health_added = C['jobs_health_added'].sum()
          C_jobs_accom_food_added = C['jobs_accom_food_added'].sum()
          C_jobs_manuf_added = C['jobs_manuf_added'].sum()
          C_jobs_office_added = C['jobs_office_added'].sum()
          C_jobs_other_added = C['jobs_other_added'].sum()
          C_jobs_retail_added = C['jobs_retail_added'].sum()
          C_jobs_wholesale_added = C['jobs_wholesale_added'].sum()

          C_new_retail_jobs = C_jobs_accom_food_added + C_jobs_retail_added
          C_new_industrial_jobs = C_jobs_manuf_added + C_jobs_wholesale_added
          C_new_office_jobs = C_jobs_other_added + C_jobs_office_added + C_jobs_health_added + C_gov_edu_added
          C_new_jobs = C_new_retail_jobs + C_new_industrial_jobs + C_new_office_jobs

          # new jobs (non-centered)
          NC_gov_edu_added = NC['jobs_gov_edu_added'].sum()
          NC_jobs_health_added = NC['jobs_health_added'].sum()
          NC_jobs_accom_food_added = NC['jobs_accom_food_added'].sum()
          NC_jobs_manuf_added = NC['jobs_manuf_added'].sum()
          NC_jobs_office_added = NC['jobs_office_added'].sum()
          NC_jobs_other_added = NC['jobs_other_added'].sum()
          NC_jobs_retail_added = NC['jobs_retail_added'].sum()
          NC_jobs_wholesale_added = NC['jobs_wholesale_added'].sum()

          NC_new_retail_jobs = NC_jobs_accom_food_added + NC_jobs_retail_added
          NC_new_industrial_jobs = NC_jobs_manuf_added + NC_jobs_wholesale_added
          NC_new_office_jobs = NC_jobs_other_added + NC_jobs_office_added + NC_jobs_health_added + NC_gov_edu_added
          NC_new_jobs = NC_new_retail_jobs + NC_new_industrial_jobs + NC_new_office_jobs

          # filter to new residential
          C_new_sf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_sf']==1))]
          C_new_mf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_mf']==1))]
          NC_new_sf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_sf']==1))]
          NC_new_mf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_mf']==1))]

          # get the totals & counts
          C_new_sf_units = C_new_sf['residential_units'].sum()
          C_new_mf_units = C_new_mf['residential_units'].sum()
          NC_new_sf_units = NC_new_sf['residential_units'].sum()
          NC_new_mf_units = NC_new_mf['residential_units'].sum()
          C_new_units = C_new_sf_units + C_new_mf_units
          NC_new_units = NC_new_sf_units + NC_new_mf_units

          #---------------------------
          # Greenfield(G) vs Redev(R)
          #---------------------------

          # separate into Greenfield and Redev
          G = df[df['was_developed']==1].copy()
          R = df[df['was_redeveloped']==1].copy()

          # new jobs (Greenfield)
          G_gov_edu_added = G['jobs_gov_edu_added'].sum()
          G_jobs_health_added = G['jobs_health_added'].sum()
          G_jobs_accom_food_added = G['jobs_accom_food_added'].sum()
          G_jobs_manuf_added = G['jobs_manuf_added'].sum()
          G_jobs_office_added = G['jobs_office_added'].sum()
          G_jobs_other_added = G['jobs_other_added'].sum()
          G_jobs_retail_added = G['jobs_retail_added'].sum()
          G_jobs_wholesale_added = G['jobs_wholesale_added'].sum()

          G_new_retail_jobs = G_jobs_accom_food_added + G_jobs_retail_added
          G_new_industrial_jobs = G_jobs_manuf_added + G_jobs_wholesale_added
          G_new_office_jobs = G_jobs_other_added + G_jobs_office_added + G_jobs_health_added + G_gov_edu_added
          G_new_jobs = G_new_retail_jobs + G_new_industrial_jobs + G_new_office_jobs

          # new jobs (non-Redev)
          R_gov_edu_added = R['jobs_gov_edu_added'].sum()
          R_jobs_health_added = R['jobs_health_added'].sum()
          R_jobs_accom_food_added = R['jobs_accom_food_added'].sum()
          R_jobs_manuf_added = R['jobs_manuf_added'].sum()
          R_jobs_office_added = R['jobs_office_added'].sum()
          R_jobs_other_added = R['jobs_other_added'].sum()
          R_jobs_retail_added = R['jobs_retail_added'].sum()
          R_jobs_wholesale_added = R['jobs_wholesale_added'].sum()

          R_new_retail_jobs = R_jobs_accom_food_added + R_jobs_retail_added
          R_new_industrial_jobs = R_jobs_manuf_added + R_jobs_wholesale_added
          R_new_office_jobs = R_jobs_other_added + R_jobs_office_added + R_jobs_health_added + R_gov_edu_added
          R_new_jobs = R_new_retail_jobs + R_new_industrial_jobs + R_new_office_jobs

          # filter to new residential
          G_new_sf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_sf']==1))]
          G_new_mf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_mf']==1))]
          R_new_sf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_sf']==1))]
          R_new_mf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_mf']==1))]

          # get the totals & counts
          G_new_sf_units = G_new_sf['residential_units'].sum()
          G_new_mf_units = G_new_mf['residential_units'].sum()
          R_new_sf_units = R_new_sf['residential_units'].sum()
          R_new_mf_units = R_new_mf['residential_units'].sum()
          G_new_units = G_new_sf_units + G_new_mf_units
          R_new_units = R_new_sf_units + R_new_mf_units


          # append current data to final dataframe 
          d = {
               'year': [year],

               'C_new_res_units': [C_new_units], 'NC_new_res_units': [NC_new_units], 
               'C_new_sf_units': [C_new_sf_units], 'NC_new_sf_units': [NC_new_sf_units],
               'C_new_mf_units': [C_new_mf_units], 'NC_new_mf_units': [NC_new_mf_units],
               'C_new_jobs':[C_new_jobs], 'NC_new_jobs':[NC_new_jobs],
               'C_new_retail_jobs':[C_new_retail_jobs], 'NC_new_retail_jobs':[NC_new_retail_jobs], 
               'C_new_industrial_jobs':[C_new_industrial_jobs],'NC_new_industrial_jobs':[NC_new_industrial_jobs], 
               'C_new_office_jobs':[C_new_office_jobs], 'NC_new_office_jobs':[NC_new_office_jobs],

               'G_new_res_units': [G_new_units], 'R_new_res_units': [R_new_units], 
               'G_new_sf_units': [G_new_sf_units], 'R_new_sf_units': [R_new_sf_units],
               'G_new_mf_units': [G_new_mf_units], 'R_new_mf_units': [R_new_mf_units],
               'G_new_jobs':[G_new_jobs], 'R_new_jobs':[R_new_jobs],
               'G_new_retail_jobs':[G_new_retail_jobs], 'R_new_retail_jobs':[R_new_retail_jobs], 
               'G_new_industrial_jobs':[G_new_industrial_jobs],'R_new_industrial_jobs':[R_new_industrial_jobs], 
               'G_new_office_jobs':[G_new_office_jobs], 'R_new_office_jobs':[R_new_office_jobs]
               }

          row = pd.DataFrame(data=d)
          summary = pd.concat([summary, row]).astype(int)

          # export
          summary = summary[summary['year'] != 2019]
          city_no_space = city.replace(' ', '_')
          summary.to_csv(os.path.join(city_area_outputs, f'{city_no_space}_{name}.csv'), index=False)

KeyboardInterrupt: 

# By Center Type

In [34]:
center_type_outputs = os.path.join(outputs, 'County')
if not os.path.exists(center_type_outputs):
    os.makedirs(center_type_outputs)

center_types = list(set(eq[(eq['CENTER_TYPE'].isna()==False)]['CENTER_TYPE'].to_list()))
center_types

['Urban Center',
 'City Center',
 'Neighborhood Center',
 'Educational Center',
 'Industrial District',
 'Special District',
 'Employment District',
 'Metropolitan Center']

In [35]:
for center_type in center_types:
     
     summary = pd.DataFrame()
     for csv in csvs:
          df = pd.read_csv(csv)
          df = df.merge(eq, on='parcel_id', how='left')
          df = df[df['CENTER_TYPE']== center_type].copy()

          year = os.path.basename(csv).split('_')[3]

          #--------------------------------------
          # Centered(C) vs Noncentered (NC)
          #--------------------------------------

          # separate into centered and noncentered
          C = df[df['parcel_id'].isin(centers_eq_ids)==True].copy()
          NC = df[df['parcel_id'].isin(centers_eq_ids)==False].copy()

          # new jobs (centered)
          C_gov_edu_added = C['jobs_gov_edu_added'].sum()
          C_jobs_health_added = C['jobs_health_added'].sum()
          C_jobs_accom_food_added = C['jobs_accom_food_added'].sum()
          C_jobs_manuf_added = C['jobs_manuf_added'].sum()
          C_jobs_office_added = C['jobs_office_added'].sum()
          C_jobs_other_added = C['jobs_other_added'].sum()
          C_jobs_retail_added = C['jobs_retail_added'].sum()
          C_jobs_wholesale_added = C['jobs_wholesale_added'].sum()

          C_new_retail_jobs = C_jobs_accom_food_added + C_jobs_retail_added
          C_new_industrial_jobs = C_jobs_manuf_added + C_jobs_wholesale_added
          C_new_office_jobs = C_jobs_other_added + C_jobs_office_added + C_jobs_health_added + C_gov_edu_added
          C_new_jobs = C_new_retail_jobs + C_new_industrial_jobs + C_new_office_jobs

          # new jobs (non-centered)
          NC_gov_edu_added = NC['jobs_gov_edu_added'].sum()
          NC_jobs_health_added = NC['jobs_health_added'].sum()
          NC_jobs_accom_food_added = NC['jobs_accom_food_added'].sum()
          NC_jobs_manuf_added = NC['jobs_manuf_added'].sum()
          NC_jobs_office_added = NC['jobs_office_added'].sum()
          NC_jobs_other_added = NC['jobs_other_added'].sum()
          NC_jobs_retail_added = NC['jobs_retail_added'].sum()
          NC_jobs_wholesale_added = NC['jobs_wholesale_added'].sum()

          NC_new_retail_jobs = NC_jobs_accom_food_added + NC_jobs_retail_added
          NC_new_industrial_jobs = NC_jobs_manuf_added + NC_jobs_wholesale_added
          NC_new_office_jobs = NC_jobs_other_added + NC_jobs_office_added + NC_jobs_health_added + NC_gov_edu_added
          NC_new_jobs = NC_new_retail_jobs + NC_new_industrial_jobs + NC_new_office_jobs

          # filter to new residential
          C_new_sf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_sf']==1))]
          C_new_mf = C[((C['was_developed']==1) | (C['was_redeveloped']==1)) & ((C['is_mf']==1))]
          NC_new_sf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_sf']==1))]
          NC_new_mf = NC[((NC['was_developed']==1) | (NC['was_redeveloped']==1)) & ((NC['is_mf']==1))]

          # get the totals & counts
          C_new_sf_units = C_new_sf['residential_units'].sum()
          C_new_mf_units = C_new_mf['residential_units'].sum()
          NC_new_sf_units = NC_new_sf['residential_units'].sum()
          NC_new_mf_units = NC_new_mf['residential_units'].sum()
          C_new_units = C_new_sf_units + C_new_mf_units
          NC_new_units = NC_new_sf_units + NC_new_mf_units

          #---------------------------
          # Greenfield(G) vs Redev(R)
          #---------------------------

          # separate into Greenfield and Redev
          G = df[df['was_developed']==1].copy()
          R = df[df['was_redeveloped']==1].copy()

          # new jobs (Greenfield)
          G_gov_edu_added = G['jobs_gov_edu_added'].sum()
          G_jobs_health_added = G['jobs_health_added'].sum()
          G_jobs_accom_food_added = G['jobs_accom_food_added'].sum()
          G_jobs_manuf_added = G['jobs_manuf_added'].sum()
          G_jobs_office_added = G['jobs_office_added'].sum()
          G_jobs_other_added = G['jobs_other_added'].sum()
          G_jobs_retail_added = G['jobs_retail_added'].sum()
          G_jobs_wholesale_added = G['jobs_wholesale_added'].sum()

          G_new_retail_jobs = G_jobs_accom_food_added + G_jobs_retail_added
          G_new_industrial_jobs = G_jobs_manuf_added + G_jobs_wholesale_added
          G_new_office_jobs = G_jobs_other_added + G_jobs_office_added + G_jobs_health_added + G_gov_edu_added
          G_new_jobs = G_new_retail_jobs + G_new_industrial_jobs + G_new_office_jobs

          # new jobs (non-Redev)
          R_gov_edu_added = R['jobs_gov_edu_added'].sum()
          R_jobs_health_added = R['jobs_health_added'].sum()
          R_jobs_accom_food_added = R['jobs_accom_food_added'].sum()
          R_jobs_manuf_added = R['jobs_manuf_added'].sum()
          R_jobs_office_added = R['jobs_office_added'].sum()
          R_jobs_other_added = R['jobs_other_added'].sum()
          R_jobs_retail_added = R['jobs_retail_added'].sum()
          R_jobs_wholesale_added = R['jobs_wholesale_added'].sum()

          R_new_retail_jobs = R_jobs_accom_food_added + R_jobs_retail_added
          R_new_industrial_jobs = R_jobs_manuf_added + R_jobs_wholesale_added
          R_new_office_jobs = R_jobs_other_added + R_jobs_office_added + R_jobs_health_added + R_gov_edu_added
          R_new_jobs = R_new_retail_jobs + R_new_industrial_jobs + R_new_office_jobs

          # filter to new residential
          G_new_sf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_sf']==1))]
          G_new_mf = G[((G['was_developed']==1) | (G['was_redeveloped']==1)) & ((G['is_mf']==1))]
          R_new_sf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_sf']==1))]
          R_new_mf = R[((R['was_developed']==1) | (R['was_redeveloped']==1)) & ((R['is_mf']==1))]

          # get the totals & counts
          G_new_sf_units = G_new_sf['residential_units'].sum()
          G_new_mf_units = G_new_mf['residential_units'].sum()
          R_new_sf_units = R_new_sf['residential_units'].sum()
          R_new_mf_units = R_new_mf['residential_units'].sum()
          G_new_units = G_new_sf_units + G_new_mf_units
          R_new_units = R_new_sf_units + R_new_mf_units


          # append current data to final dataframe 
          d = {
               'year': [year],

               'C_new_res_units': [C_new_units], 'NC_new_res_units': [NC_new_units], 
               'C_new_sf_units': [C_new_sf_units], 'NC_new_sf_units': [NC_new_sf_units],
               'C_new_mf_units': [C_new_mf_units], 'NC_new_mf_units': [NC_new_mf_units],
               'C_new_jobs':[C_new_jobs], 'NC_new_jobs':[NC_new_jobs],
               'C_new_retail_jobs':[C_new_retail_jobs], 'NC_new_retail_jobs':[NC_new_retail_jobs], 
               'C_new_industrial_jobs':[C_new_industrial_jobs],'NC_new_industrial_jobs':[NC_new_industrial_jobs], 
               'C_new_office_jobs':[C_new_office_jobs], 'NC_new_office_jobs':[NC_new_office_jobs],

               'G_new_res_units': [G_new_units], 'R_new_res_units': [R_new_units], 
               'G_new_sf_units': [G_new_sf_units], 'R_new_sf_units': [R_new_sf_units],
               'G_new_mf_units': [G_new_mf_units], 'R_new_mf_units': [R_new_mf_units],
               'G_new_jobs':[G_new_jobs], 'R_new_jobs':[R_new_jobs],
               'G_new_retail_jobs':[G_new_retail_jobs], 'R_new_retail_jobs':[R_new_retail_jobs], 
               'G_new_industrial_jobs':[G_new_industrial_jobs],'R_new_industrial_jobs':[R_new_industrial_jobs], 
               'G_new_office_jobs':[G_new_office_jobs], 'R_new_office_jobs':[R_new_office_jobs]
               }

          row = pd.DataFrame(data=d)
          summary = pd.concat([summary, row]).astype(int)

          # export
          summary = summary[summary['year'] != 2019]
          center_type_no_space = center_type.replace(' ', '_')
          summary.to_csv(os.path.join(center_type_outputs, f'{center_type_no_space}_{name}.csv'), index=False)